In [1]:
import torch
import torch.nn as nn
from torchmetrics.classification import BinaryJaccardIndex, JaccardIndex
from torchmetrics.functional.classification import multiclass_jaccard_index
from torchmetrics.functional.segmentation import mean_iou
from torch.nn.functional import softmax, one_hot

In [2]:
X = torch.rand(size=(1,3,3,8))
Y = (torch.rand(size=(1,3,3,8)) > 0.5) * 1

In [3]:
print(X)
print(Y)

tensor([[[[0.6372, 0.2638, 0.2604, 0.8549, 0.8723, 0.7034, 0.5103, 0.0013],
          [0.3022, 0.2188, 0.8299, 0.1832, 0.9009, 0.7847, 0.4996, 0.9875],
          [0.5134, 0.3053, 0.5092, 0.5875, 0.5072, 0.1183, 0.0286, 0.9251]],

         [[0.5064, 0.8409, 0.4776, 0.4451, 0.9370, 0.7479, 0.8464, 0.8820],
          [0.2856, 0.0868, 0.1285, 0.3299, 0.2683, 0.6146, 0.5490, 0.8278],
          [0.3428, 0.2099, 0.4739, 0.4193, 0.9067, 0.6751, 0.3898, 0.9705]],

         [[0.7600, 0.6519, 0.6302, 0.8279, 0.8793, 0.9622, 0.1932, 0.5835],
          [0.1515, 0.2482, 0.9301, 0.3756, 0.5505, 0.7285, 0.4053, 0.2314],
          [0.4934, 0.4187, 0.8079, 0.1795, 0.2323, 0.7519, 0.8169, 0.2160]]]])
tensor([[[[0, 1, 0, 1, 1, 1, 1, 0],
          [1, 1, 0, 1, 1, 0, 1, 0],
          [1, 1, 0, 1, 0, 1, 0, 1]],

         [[0, 0, 1, 1, 1, 0, 1, 0],
          [0, 1, 1, 0, 1, 0, 0, 0],
          [1, 1, 1, 1, 1, 0, 0, 0]],

         [[1, 1, 0, 0, 0, 1, 1, 0],
          [0, 1, 1, 1, 0, 1, 1, 0],
          [0, 1, 

In [4]:
m1 = BinaryJaccardIndex()
m2 = JaccardIndex(task='binary', average='micro')

In [5]:
print(m1(X,Y), m2(X,Y))

tensor(0.2903) tensor(0.2903)


In [6]:
# s = 0
# for i in range(7):
#     t = m1(X[i], Y[i])
#     s += t
#     print(t)
# print(s, s/8)

In [7]:
p1 = torch.mean(X, axis = (1,2))
print(p1)

tensor([[0.4436, 0.3605, 0.5609, 0.4670, 0.6727, 0.6763, 0.4710, 0.6250]])


In [8]:
print(torch.mean(X[0]))

tensor(0.5346)


In [9]:
# print(torch.mean(X[1]))

## Torchmetrics multiclass_jaccard_index vs mean_iou

In [10]:
pred = softmax(torch.rand((4,21,224,224)), dim=1)
target = torch.randint(21, (4,224,224))
target_oh = torch.permute(one_hot(target, num_classes = 21), (0,3,1,2))
pred_oh = torch.permute(one_hot(torch.argmax(pred, dim=1), num_classes = 21), (0,3,1,2))

In [11]:
mIU = mean_iou(torch.argmax(pred, dim=1), target, num_classes=21, per_class=True)
mIU1 = mean_iou(pred_oh, target_oh, num_classes=21, per_class=True)
iou = multiclass_jaccard_index(pred, target, num_classes = 21, average = None)
iou2 = multiclass_jaccard_index(pred, target, num_classes = 21)

In [12]:
# print('mIU', mIU)
# print('iou', iou)
# print('iou2', iou2)
# print('mIU1', mIU1)

In [13]:
torch.mean(mIU, dim=0)

tensor([0.0241, 0.0257, 0.0228, 0.0231, 0.0243, 0.0223, 0.0237, 0.0234, 0.0252,
        0.0246, 0.0230, 0.0259, 0.0251, 0.0220, 0.0241, 0.0236, 0.0235, 0.0253,
        0.0241, 0.0228, 0.0245])

In [14]:
# x = torch.zeros([21])
# print(x.shape)
# for i in range(21):
#     x[i] = torch.sum(torch.where(target == i, 1, 0))
#     print(x[i])
# print(torch.sum(x))

In [15]:
4*224*224

200704

In [18]:
nc = 5
preds = torch.randint(0, 2, (4, nc, 16, 16))  # 4 samples, 5 classes, 16x16 prediction
target = torch.randint(0, 2, (4, nc, 16, 16))  # 4 samples, 5 classes, 16x16 target
print(mean_iou(preds, target, num_classes=nc))

print(mean_iou(preds, target, num_classes=nc, per_class=True))

tensor([0.3485, 0.3396, 0.3155, 0.3226])
tensor([[0.3368, 0.3763, 0.3782, 0.3177, 0.3333],
        [0.2953, 0.3404, 0.3568, 0.3384, 0.3673],
        [0.2980, 0.3564, 0.3069, 0.3000, 0.3163],
        [0.3237, 0.4053, 0.2932, 0.2836, 0.3073]])


In [19]:
print(mean_iou(torch.argmax(preds, dim=1), torch.argmax(target, dim=1), num_classes=nc))

print(mean_iou(torch.argmax(preds, dim=1), torch.argmax(target, dim=1), num_classes=nc, per_class=True))

tensor([0.1442, 0.1256, 0.1082, 0.1045])
tensor([[0.3467, 0.1364, 0.1111, 0.0769, 0.0500],
        [0.3300, 0.1682, 0.1296, 0.0000, 0.0000],
        [0.3284, 0.1491, 0.0278, 0.0357, 0.0000],
        [0.3594, 0.1204, 0.0426, 0.0000, 0.0000]])


In [20]:
multiclass_jaccard_index(preds, target, num_classes = 5, average = None)

tensor([0.3226, 0.3312, 0.0000, 0.0000, 0.0000])